## Ejercicio 2: Manejo de múltiples clientes
Modifica el servidor basado en asyncio para que pueda manejar múltiples clientes simultáneamente. Cada cliente debería ser capaz de enviar múltiples solicitudes sin esperar que las anteriores se completen.

Pasos:

Modifica el código del servidor para que pueda manejar distintas solicitudes de múltiples clientes de forma asíncrona.
Asegúrate de que el servidor pueda gestionar y mantener el estado de cada cliente por separado.
Prueba la capacidad del servidor conectando varios clientes al mismo tiempo y realizando diferentes solicitudes.

##servidor

In [1]:
import asyncio
import pickle
import nest_asyncio
nest_asyncio.apply()

# Diccionario para almacenar el estado de cada cliente
client_states = {}

async def handle_submit_job(reader, writer):
    client_id = id(writer)
    job_id = client_states.get(client_id, [0])[-1] + 1
    client_states[client_id] = client_states.get(client_id, []) + [job_id]
    writer.write(job_id.to_bytes(4, 'little'))
    await writer.drain()
    data_length = int.from_bytes(await reader.read(4), 'little')
    data = pickle.loads(await reader.read(data_length))
    result = sum(data)
    client_states[client_id][-1] = result

async def handle_get_results(reader, writer):
    client_id = id(writer)
    job_id = int.from_bytes(await reader.read(4), 'little')
    result = client_states[client_id][job_id - 1]
    result_data = pickle.dumps(result)
    writer.write(len(result_data).to_bytes(4, 'little'))
    writer.write(result_data)
    await writer.drain()

async def accept_requests(reader, writer):
    op = await reader.read(1)
    if op[0] == 0:
        await handle_submit_job(reader, writer)
    elif op[0] == 1:
        await handle_get_results(reader, writer)

async def main():
    server = await asyncio.start_server(accept_requests, '127.0.0.1', 1936)
    async with server:
        await server.serve_forever()

asyncio.run(main())
